In [2]:
!wget https://raw.githubusercontent.com/santoshdahal2016/GPT-from-scratch/main/munamadan.txt

--2023-02-11 14:48:39--  https://raw.githubusercontent.com/santoshdahal2016/GPT-from-scratch/main/munamadan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108544 (106K) [text/plain]
Saving to: ‘munamadan.txt’

munamadan.txt       100%[===================>] 106.00K  --.-KB/s    in 0.02s   

2023-02-11 14:48:40 (5.09 MB/s) - ‘munamadan.txt’ saved [108544/108544]



In [3]:
f = open('munamadan.txt', 'r')
text = f.read()

print(text[:100])

characters = sorted(set(list(text)))

print(characters)

है मेरा भाइ ! हे मेरा बहिनी ! पहाड शहरमा,
यो यौटा मैरो भक्तिको माला चढाएँ चरणमा ।
अरू ता मैले के गर्
['\n', ' ', '!', '"', '&', "'", ',', '-', '.', '1', '4', ':', ';', '?', '|', '«', '»', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ए', 'ऐ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', '्', '॑', '॒', '।', '॥', '०', '३', '७', '\u200c', '“', '”']


In [18]:
stoi = { ch:i for i , ch in enumerate(characters)}
iots = { i:ch for i , ch in enumerate(characters)}

encode = lambda s: [ stoi[c] for c in s]
decode = lambda i: ''.join([ iots[c] for c in i])

In [20]:
print(encode("है मेरा भाइ"))
# print(decode(0))

print(iots[0])

[61, 69, 1, 53, 68, 55, 62, 1, 52, 62, 22]




In [6]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)




torch.Size([42166]) torch.int64


In [7]:
n = int(0.9 * len(data))

train_data = data[:n]
val_data = data[n:]

print(train_data[:20])



tensor([61, 69,  1, 53, 68, 55, 62,  1, 52, 62, 22,  1,  2,  1, 61, 68,  1, 53,
        68, 55])


In [8]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  print(x[:t+1] , y[t])

tensor([61]) tensor(69)
tensor([61, 69]) tensor(1)
tensor([61, 69,  1]) tensor(53)
tensor([61, 69,  1, 53]) tensor(68)
tensor([61, 69,  1, 53, 68]) tensor(55)
tensor([61, 69,  1, 53, 68, 55]) tensor(62)
tensor([61, 69,  1, 53, 68, 55, 62]) tensor(1)
tensor([61, 69,  1, 53, 68, 55, 62,  1]) tensor(52)


In [9]:
batch_size = 4

def get_batch(split):
  data =  train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack( [ data[i:i+block_size] for i in ix])
  y = torch.stack( [ data[i+1:i+block_size+1] for i in ix])
  
  return x, y


In [10]:
xa,ya = get_batch('train')

print(xa.shape, ya.shape)

torch.Size([4, 8]) torch.Size([4, 8])


In [11]:
for i in range(batch_size):
  for j in range(block_size):
    print(xa[i ,:j + 1] , ya[i , j])


tensor([63]) tensor(49)
tensor([63, 49]) tensor(53)
tensor([63, 49, 53]) tensor(62)
tensor([63, 49, 53, 62]) tensor(6)
tensor([63, 49, 53, 62,  6]) tensor(0)
tensor([63, 49, 53, 62,  6,  0]) tensor(57)
tensor([63, 49, 53, 62,  6,  0, 57]) tensor(69)
tensor([63, 49, 53, 62,  6,  0, 57, 69]) tensor(56)
tensor([56]) tensor(70)
tensor([56, 70]) tensor(1)
tensor([56, 70,  1]) tensor(50)
tensor([56, 70,  1, 50]) tensor(65)
tensor([56, 70,  1, 50, 65]) tensor(55)
tensor([56, 70,  1, 50, 65, 55]) tensor(72)
tensor([56, 70,  1, 50, 65, 55, 72]) tensor(50)
tensor([56, 70,  1, 50, 65, 55, 72, 50]) tensor(65)
tensor([62]) tensor(33)
tensor([62, 33]) tensor(72)
tensor([62, 33, 72]) tensor(80)
tensor([62, 33, 72, 80]) tensor(1)
tensor([62, 33, 72, 80,  1]) tensor(34)
tensor([62, 33, 72, 80,  1, 34]) tensor(53)
tensor([62, 33, 72, 80,  1, 34, 53]) tensor(72)
tensor([62, 33, 72, 80,  1, 34, 53, 72]) tensor(29)
tensor([1]) tensor(45)
tensor([ 1, 45]) tensor(64)
tensor([ 1, 45, 64]) tensor(1)
tensor([ 1

In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BiagramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)



  def forward(self, idx ,targets=None):

    logits = self.token_embedding_table(idx)

    if targets == None:
      loss = None
    else: 
      B , T , C = logits.shape

      logits = logits.view(B*T , C)
      targets = targets.view(B*T)

      loss = F.cross_entropy(logits , targets)

    return logits , loss

  def generate(self , idx , max_new_tokens):

    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim = -1)

      idx_next = torch.multinomial(probs, num_samples=1)

      idx = torch.cat((idx, idx_next), dim=1)


    return idx



vocab_size = len(characters)
m = BiagramLanguageModel(vocab_size)

logits , loss = m(xa, ya)

print(logits.shape, loss )

# print(decode(torch.zeros((1, 1), dtype=torch.long)))

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 83]) tensor(4.8413, grad_fn=<NllLossBackward0>)

णरऔट,ं!!फिथ"०»|भँठ|७शढई"ईत्;ँंध:फं:य॥डई,तणौआङँइ'श«य1क4ग ए
”॒1ए,बकः्खटजथय३1।ण1ृ
ै॒षय:व'ौ०
आ'शठबौ4च,॒ढ


In [33]:

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
for steps in range(10000): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5181121826171875


In [40]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


छुखकोपा बूनिलरी!
क हेरोलड्ननी जी सबती गै,
चौ हल्‌ब्‌ सोरा,
न्जन त,
मी
सौ भनेर ज्न्‌ तिमनेखाङ्नकाचन अँफूर,
छै ! तियारप्जी प्दै माढा भै नैनमाकेरी,
रो,
प्यो लेशमाथेटा सहुट्‌ साईश्थली फु्यो उनैनमाउनहेको,
मौ साराले-धायौथिबिमाले तली भिलेखा नैनैनलेर,
भनै त्कालेरासावज हे,
को वपायजा! सन्छै श ।”
माकोमाइरतीपि पानैमुदेरकने न्र न तीत्गयो तिलोचागमाँडासुन, भिँधी, ?”
वी,
ख्र्रकोटा छो र भन, नलैलको आयोई भइनैशा नेप्‌ म भुने नायनजमा ह चमनीहा !”

आमनछि छ दार त्दहौ ति दुँसा,
प्कोमनैलाई थैँडी आँकोस्काल्तै भनक्र्छ छ,
आ
